In [1]:
# !pip install censusgeocode

#!pip install requests_cache


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import glob
import json
import requests
import pandas as pd
from pprint import pprint

# show all columns
pd.set_option('display.max_columns', None)

# Census Examples 

This notebook uses the `censusgeocode` package in Python (which is simply a wrapper around the US Census' official Geocoder API) to get census geographies for list of addresses or lat/longs

- https://pypi.org/project/censusgeocode/

In [10]:
df.dtypes

Unnamed: 0                     int64
CRASH DATE                    object
CRASH TIME                    object
BOROUGH                       object
zipcode                      float64
latitude                     float64
longitude                    float64
LOCATION                      object
ON STREET NAME                object
CROSS STREET NAME             object
OFF STREET NAME               object
NUMBER OF PERSONS INJURED    float64
NUMBER OF PERSONS KILLED     float64
address                       object
address_full                  object
Unnamed: 15                   object
Unnamed: 16                   object
Unnamed: 17                   object
dtype: object

### Step 1 | Grab your data at the address level

In [11]:
df = pd.read_csv('all_traffic_crashes_between_5pm_7am_geocoded.csv').query("latitude.notna()").query("longitude.notna()") # ignore two rows where lat/long are nil
#df.query("latitude.isnull()")
df.head()

/var/folders/x2/ld74q9c55_j9mn_p0hkl_0640000gn/T/ipykernel_7246/2271965160.py:1: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_traffic_crashes_between_5pm_7am_geocoded.csv').query("latitude.notna()").query("longitude.notna()") # ignore two rows where lat/long are nil


,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,zipcode,latitude,longitude,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,address,address_full,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,11,03/20/2018,2023-05-01 20:00:00,BROOKLYN,11229.0,40.610947,-73.953606,"(40.610947, -73.953606)",AVENUE P,OCEAN AVENUE,NaN,0.0,0.0,AVENUE P & OCEAN AVENUE,"AVENUE P & OCEAN AVENUE, BROOKLYN",NaN,NaN,NaN
1,14,03/20/2018,2023-05-01 21:00:00,BROOKLYN,11226.0,40.648720,-73.964910,"(40.64872, -73.96491)",CHURCH AVENUE,EAST 16 STREET,NaN,0.0,0.0,CHURCH AVENUE & EAST 16 STREET,"CHURCH AVENUE & EAST 16 STREET, BROOKLYN",NaN,NaN,NaN
2,21,03/20/2018,2023-05-01 20:18:00,MANHATTAN,10016.0,40.749350,-73.982770,"(40.74935, -73.98277)",NaN,NaN,12 EAST 36 STREET,0.0,0.0,12 EAST 36 STREET,"12 EAST 36 STREET, MANHATTAN",NaN,NaN,NaN
3,23,03/20/2018,2023-05-01 20:20:00,QUEENS,11420.0,40.675083,-73.809586,"(40.675083, -73.809586)",NaN,NaN,128-24 ROCKAWAY BOULEVARD,0.0,0.0,128-24 ROCKAWAY BOULEVARD,"128-24 ROCKAWAY BOULEVARD, QUEENS",NaN,NaN,NaN
4,26,03/20/2018,2023-05-01 06:50:00,STATEN ISLAND,10306.0,40.569893,-74.110344,"(40.569893, -74.110344)",HYLAN BOULEVARD,CODDINGTON AVENUE,NaN,0.0,0.0,HYLAN BOULEVARD & CODDINGTON AVENUE,"HYLAN BOULEVARD & CODDINGTON AVENUE, STATEN IS...",NaN,NaN,NaN


In [12]:
print(df[['latitude', 'longitude']].isna().sum())


latitude     0
longitude    0
dtype: int64


In [13]:
missing_coords = df[(df['latitude'] == 0.0) & (df['longitude'] == 0.0)]
print(missing_coords)

        Unnamed: 0  CRASH DATE           CRASH TIME    BOROUGH  zipcode  \
1779          6302  03/30/2018  2023-05-01 01:30:00  MANHATTAN  10065.0   
1885          6624  03/30/2018  2023-05-01 17:10:00  MANHATTAN  10001.0   
4509         15550  04/15/2018  2023-05-01 02:30:00   BROOKLYN  11233.0   
6257         22023  04/25/2018  2023-05-01 06:00:00     QUEENS  11004.0   
6320         22283  04/25/2018  2023-05-01 05:55:00     QUEENS  11378.0   
...            ...         ...                  ...        ...      ...   
229367      733439  02/08/2023  2023-05-01 22:30:00   BROOKLYN  11214.0   
229704      734411  02/12/2023  2023-05-01 18:06:00        NaN      NaN   
231254      738987  03/03/2023  2023-05-01 18:43:00  MANHATTAN  10016.0   
231592      739864  03/06/2023  2023-05-01 17:38:00  MANHATTAN  10035.0   
232098      741458  03/13/2023  2023-05-01 06:24:00        NaN      NaN   

        latitude  longitude    LOCATION                    ON STREET NAME  \
1779         0.0      

In [14]:
df = df.drop(missing_coords.index)

### Step 2 | Geoode Lat/Long if they're not already present

It already exists in this dataset. Census geocode has a function to go from addresss --> lat/long, but I haven't had time to implement it here. This dataset already has lat/longs. Message me if you're struggling with this step.

### Step 3 | Get Census Geographies

In [15]:
# Code adapted from:
# https://gis.stackexchange.com/questions/363830/applying-the-censusgeocode-package-to-an-entire-dataframe-of-geocoded-data
# Defines a geocode function that accepts lat/long and spits out geographies
# The code then runs that funciton in parllel (for speed).

import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        #latitudes = df['lat']
        #longitudes = df['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df)))

    return pd.DataFrame(data)

traffic_crashes_census = bulk_geocode(df['latitude'], df['longitude']) 
traffic_crashes_census.head()

  0%|          | 0/246228 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,INTPTLAT,FUNCSTAT,NAME,OBJECTID,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,73,360470550003001,+40.6106592,3001,0,36,3001,210701004655659,BK,+40.6106592,S,Block 3001,4377017,055000,-073.9542175,3,4884,24,-073.9542175,G5040,L,U,047
1,,153,360470506002001,+40.6495095,2001,0,36,2001,210701004652861,BK,+40.6495095,S,Block 2001,4262189,050600,-073.9655236,2,17083,45,-073.9655236,G5040,L,U,047
2,,242,360610082002007,+40.7490774,2007,0,36,2007,210701008621168,BK,+40.7490774,S,Block 2007,2885753,008200,-073.9830496,2,12152,162,-073.9830496,G5040,L,U,061
3,,67,360810818003000,+40.6746477,3000,0,36,3000,210701006107204,BK,+40.6746477,S,Block 3000,2012060,081800,-073.8091830,3,10503,30,-073.8091830,G5040,L,U,081
4,,68,360850134001022,+40.5701759,1022,0,36,1022,210701004605017,BK,+40.5701759,S,Block 1022,5406060,013400,-074.1116816,1,17262,26,-074.1116816,G5040,L,U,085


In [17]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
traffic_crashes_census = traffic_crashes_census[to_keep]
traffic_crashes_census

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360470550003001,36,047,055000,3001
1,360470506002001,36,047,050600,2001
2,360610082002007,36,061,008200,2007
3,360810818003000,36,081,081800,3000
4,360850134001022,36,085,013400,1022
...,...,...,...,...,...
246223,360810198001001,36,081,019800,1001
246224,360470192002002,36,047,019200,2002
246225,360810803012005,36,081,080301,2005
246226,360811417005001,36,081,141700,5001


In [18]:
traffic_crashes_census = pd.concat(
    [ 
        df.reset_index(drop=True),
     traffic_crashes_census.reset_index(drop=True)
    ], 
    axis=1)

traffic_crashes_census.head()

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,zipcode,latitude,longitude,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,address,address_full,Unnamed: 15,Unnamed: 16,Unnamed: 17,GEOID,STATE,COUNTY,TRACT,BLOCK
0,11,03/20/2018,2023-05-01 20:00:00,BROOKLYN,11229.0,40.610947,-73.953606,"(40.610947, -73.953606)",AVENUE P,OCEAN AVENUE,NaN,0.0,0.0,AVENUE P & OCEAN AVENUE,"AVENUE P & OCEAN AVENUE, BROOKLYN",NaN,NaN,NaN,360470550003001,36,047,055000,3001
1,14,03/20/2018,2023-05-01 21:00:00,BROOKLYN,11226.0,40.648720,-73.964910,"(40.64872, -73.96491)",CHURCH AVENUE,EAST 16 STREET,NaN,0.0,0.0,CHURCH AVENUE & EAST 16 STREET,"CHURCH AVENUE & EAST 16 STREET, BROOKLYN",NaN,NaN,NaN,360470506002001,36,047,050600,2001
2,21,03/20/2018,2023-05-01 20:18:00,MANHATTAN,10016.0,40.749350,-73.982770,"(40.74935, -73.98277)",NaN,NaN,12 EAST 36 STREET,0.0,0.0,12 EAST 36 STREET,"12 EAST 36 STREET, MANHATTAN",NaN,NaN,NaN,360610082002007,36,061,008200,2007
3,23,03/20/2018,2023-05-01 20:20:00,QUEENS,11420.0,40.675083,-73.809586,"(40.675083, -73.809586)",NaN,NaN,128-24 ROCKAWAY BOULEVARD,0.0,0.0,128-24 ROCKAWAY BOULEVARD,"128-24 ROCKAWAY BOULEVARD, QUEENS",NaN,NaN,NaN,360810818003000,36,081,081800,3000
4,26,03/20/2018,2023-05-01 06:50:00,STATEN ISLAND,10306.0,40.569893,-74.110344,"(40.569893, -74.110344)",HYLAN BOULEVARD,CODDINGTON AVENUE,NaN,0.0,0.0,HYLAN BOULEVARD & CODDINGTON AVENUE,"HYLAN BOULEVARD & CODDINGTON AVENUE, STATEN IS...",NaN,NaN,NaN,360850134001022,36,085,013400,1022


In [20]:
len(traffic_crashes_census)

246228

In [ ]:
geocoded_crashes_df.csv

In [19]:
traffic_crashes_census.to_csv('traffic_crashes_census.csv', index=False)

# Step 4 | Pick a geographical level and get Census data
Do you want Census data at the state level? county? tract? block?

1. Pick a geographical level.
2. See `census-example.ipynb` if you want to learn how to get Census data at your desired level

# Hope that helps!